<a href="https://colab.research.google.com/github/ntolayd/Projects/blob/main/Recommendation_Engines/Train_Test_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!unzip "/content/gdrive/My Drive/ml-1m.zip"

Archive:  /content/gdrive/My Drive/ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [17]:
import pandas as pd
import numpy as np
import pickle

In [5]:
movies = pd.read_csv("ml-1m/movies.dat", sep='::', engine='python', header=None)
movies.columns = ['movie_id', 'movie_name', 'genre']
ratings = pd.read_csv("ml-1m/ratings.dat", sep='::', engine='python', header=None)
ratings.columns=['user_id', 'movie_id', 'rating','timestamp']
ratings.drop('timestamp',axis=1, inplace=True)

In [7]:
#converting to user - item matrix
train_matrix = pd.pivot_table(ratings, index='user_id', columns='movie_id', values='rating')
arr = train_matrix.copy().values

In [8]:
#replacing random selected values with 0 for validation
def replace(orig, percentage=0.2):
  """
  Replaces 'percentage'% of the original values in 'orig' with 0's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  np.random.seed(35)
  new_data = orig.copy()
  rated = np.where(~np.isnan(arr))
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 0
  return new_data, (rated[0][idx], rated[1][idx])

In [9]:
masked, masked_idx = replace(arr)
masked = pd.DataFrame(masked, index=train_matrix.index, columns=train_matrix.columns)

In [10]:
masked_melted = pd.melt(masked.reset_index(), id_vars=['user_id']).dropna()

In [11]:
test_set = masked_melted[masked_melted['value'] == 0]
test_set = pd.merge(test_set, ratings, on=['user_id', 'movie_id'])

In [20]:
#writing files to google drive
masked.to_csv('masked.csv')
!cp masked.csv "gdrive/My Drive/"

masked_melted.to_csv('masked_melted.csv')
!cp masked_melted.csv "gdrive/My Drive/"

test_set.to_csv('test_set.csv')
!cp test_set.csv "gdrive/My Drive/"

with open('masked_idx.pkl', 'wb') as fid:
     pickle.dump(masked_idx, fid)
!cp masked_idx.pkl "gdrive/My Drive/"